In [15]:
# Import findspark and initialize. 
import findspark
findspark.init()

In [16]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [17]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)

df.printSchema()

24/06/30 18:50:23 WARN SparkContext: The path https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv has been added already. Overwriting of added paths is not supported in the current version.


root
 |-- id: string (nullable = true)
 |-- date: date (nullable = true)
 |-- date_built: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- bathrooms: integer (nullable = true)
 |-- sqft_living: integer (nullable = true)
 |-- sqft_lot: integer (nullable = true)
 |-- floors: integer (nullable = true)
 |-- waterfront: integer (nullable = true)
 |-- view: integer (nullable = true)



In [19]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView("home_sales")



In [20]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
query_1 = """
SELECT YEAR(date) AS year, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 4
GROUP BY YEAR(date)
ORDER BY YEAR(date)
"""
result_1 = spark.sql(query_1)
result_1.show()


+----+---------+
|year|avg_price|
+----+---------+
|2019| 300263.7|
|2020|298353.78|
|2021|301819.44|
|2022|296363.88|
+----+---------+



In [24]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
# Adjusted query to directly use integer value for grouping by year
query_2 = """
SELECT date_built AS year_built, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3
GROUP BY date_built
ORDER BY year_built
"""
result_2 = spark.sql(query_2)
result_2.show()




+----------+---------+
|year_built|avg_price|
+----------+---------+
|      2010|292859.62|
|      2011|291117.47|
|      2012|293683.19|
|      2013|295962.27|
|      2014|290852.27|
|      2015| 288770.3|
|      2016|290555.07|
|      2017|292676.79|
+----------+---------+



In [25]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?

query_3 = """
SELECT date_built AS year_built, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
GROUP BY date_built
ORDER BY year_built
"""
result_3 = spark.sql(query_3)
result_3.show()


+----------+---------+
|year_built|avg_price|
+----------+---------+
|      2010|285010.22|
|      2011|276553.81|
|      2012|307539.97|
|      2013|303676.79|
|      2014|298264.72|
|      2015|297609.97|
|      2016| 293965.1|
|      2017|280317.58|
+----------+---------+



In [26]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating. 
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()

query_4 = """
SELECT view, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
GROUP BY view
HAVING AVG(price) >= 350000
ORDER BY view DESC
"""
result_4 = spark.sql(query_4)
result_4.show()

end_time = time.time()
print(f"Runtime: {end_time - start_time:.2f} seconds")

+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

Runtime: 1.26 seconds


In [27]:
# 7. Cache the the temporary table home_sales.
spark.catalog.cacheTable("home_sales")

# Check if the table is cached
is_cached = spark.catalog.isCached("home_sales")
print(f"Is 'home_sales' table cached? {is_cached}")

Is 'home_sales' table cached? True


In [28]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [29]:
# 9. Using the cached data, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the uncached runtime.

# Using the cached data, run the query
start_time_cached = time.time()

result_4_cached = spark.sql(query_4)
result_4_cached.show()

end_time_cached = time.time()
runtime_cached = end_time_cached - start_time_cached
print(f"Runtime with cache: {runtime_cached:.2f} seconds")

# Compare with uncached runtime
print(f"Uncached Runtime: {end_time - start_time:.2f} seconds")
print(f"Speedup with caching: {((end_time - start_time) - runtime_cached) / (end_time - start_time) * 100:.2f}%")



+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

Runtime with cache: 1.37 seconds
Uncached Runtime: 1.26 seconds
Speedup with caching: -8.38%


In [30]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 

df.write.partitionBy("date_built").parquet("path_to_output_directory/home_sales_partitioned.parquet")


In [31]:
# 11. Read the formatted parquet data.
parquet_path = "path_to_output_directory/home_sales_partitioned.parquet"

# Read partitioned Parquet data into a DataFrame
partitioned_df = spark.read.parquet(parquet_path)

# Create a temporary view for the partitioned DataFrame
partitioned_df.createOrReplaceTempView("partitioned_home_sales")


In [32]:
# 12. Create a temporary table for the parquet data.
partitioned_df.createOrReplaceTempView("partitioned_home_sales_temp")


In [33]:
# 13. Using the parquet DataFrame, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the cached runtime.

# Using the partitioned Parquet DataFrame, run the query
start_time_parquet = time.time()

query_4_parquet = """
SELECT view, ROUND(AVG(price), 2) AS avg_price
FROM partitioned_home_sales_temp
GROUP BY view
HAVING AVG(price) >= 350000
ORDER BY view DESC
"""
result_4_parquet = spark.sql(query_4_parquet)
result_4_parquet.show()

end_time_parquet = time.time()
runtime_parquet = end_time_parquet - start_time_parquet
print(f"Runtime with Parquet: {runtime_parquet:.2f} seconds")

# Compare with cached runtime
print(f"Cached Runtime: {runtime_cached:.2f} seconds")
print(f"Speedup with Parquet: {((runtime_cached - runtime_parquet) / runtime_cached) * 100:.2f}%")


+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

Runtime with Parquet: 1.26 seconds
Cached Runtime: 1.37 seconds
Speedup with Parquet: 7.55%


In [34]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_sales")

# Verify that the table is uncached
is_cached = spark.catalog.isCached("home_sales")
print(f"Is 'home_sales' table cached? {is_cached}")


Is 'home_sales' table cached? False


In [35]:
# 15. Check if the home_sales is no longer cached
is_cached = spark.catalog.isCached("home_sales")
print(f"Is 'home_sales' table cached? {is_cached}")


Is 'home_sales' table cached? False
